# __User Tasks__
The frontend will also handle a lot of validation and error handling, so pretend this is the best-case scenario. We can walk through the logical order of things on the frontend. 
The frontend will also have to handle multi-user activity, but only in so far as what user_id variable is currently active, etc. Simple things. As long as Firebase functions can handle simultaneous activity, we're good. 

In [1]:
# Setup 

import os
import sys
from dotenv import load_dotenv
from firebase_admin import credentials
import firebase_admin

# Add project root and utils to path
sys.path.append('/Users/geneva/Downloads/critics')
sys.path.append('/Users/geneva/Downloads/critics/utils')

from utils.user import *
from utils.board import *
from utils.firebase import *
from utils.restaurant import *
from utils.category import *

# Load environment variables
load_dotenv()

def initialize_firebase():
    """Initialize Firebase connection using credentials from environment"""
    service_account_path = os.getenv('SERVICE_ACCOUNT_PATH')
    database_url = os.getenv('DATABASE_URL')

    if not service_account_path:
        raise ValueError("SERVICE_ACCOUNT_PATH environment variable is not set.")

    if not firebase_admin._apps:
        cred = credentials.Certificate(service_account_path)
        firebase_admin.initialize_app(cred, {
            'databaseURL': database_url
        })

# Set service account path and initialize
os.environ['SERVICE_ACCOUNT_PATH'] = '/Users/geneva/Downloads/critics/gcreds_test.json'
initialize_firebase()



In [4]:
# New user is created when they first open the app. 

name = "Geneva"
user_id = "user_001"  

new_user = create_user(name, user_id)
print(f"Created new user: {new_user}")

Created new user: {'name': 'Geneva', 'boards': [], 'last_active': None}


In [7]:
# User creates a new board, and names it "This is a test board"

board_id = "board_001"  
board_name = "Genny's Board"

#create the board 
new_board = create_board(board_id, board_name, user_id) # user_id from the prev. block

# add the board to the user's boards
updated_boards = join_board(user_id, board_id)

# Add the user as a member of the board
add_member_to_board(board_id, user_id)

print(f"Created new board: {new_board}")


Created new board: {'name': "Genny's Board", 'categories': {}, 'members': [], 'owner': 'user_001'}


In [9]:
# New user is added, then joins the board. 

name = "Chris"
user_id = "user_002"  

new_user = create_user(name, user_id)
print(f"Created new user: {new_user}")

# Add this board to chris's boards
updated_boards = join_board(user_id, board_id)  # board_id from previous cell

# add chris to the list of board's members
add_member_to_board(board_id, user_id)

print(f"Added {name} to board: {board_id}")


Created new user: {'name': 'Chris', 'boards': [], 'last_active': None}
Added Chris to board: board_001


In [2]:
# Geneva adds a category to the board.

board_id = "board_001" 
category_id = "category_001"
category_name = "Fine Dining"
caption = "Can't tell if we go to dine, or to dress up."

# create the category
new_category = add_category(category_id, category_name, caption)

# add the category to the board's list of categories
add_category_to_board(board_id, category_id)

In [2]:
# Geneva adds a restaurant to the board. 

category_id = "category_001" # Fine Dining 
restaurant_id = "restaurant_001" 

data = {
    "name": "The Modern",
    "rating_1": 9,
    "rating_2": 8,
    "rating_3": 7,
    "notes": "The food was good, but the service was a bit slow.",
    "visits": ["2023-05-15", "2023-07-22", "2023-09-03"],
    "location": "9 W 57th St, New York, NY 10019",
    "dishes": ["Roasted Beet Salad", "Filet Mignon", "Chocolate Cake"],
    "photo": "https://www.themodern.com/images/the-modern-restaurant-exterior.jpg"
}

new_restaurant = add_restaurant(category_id, restaurant_id, data)

In [4]:
# Chris adds a restaurant to the board. 

category_id = "category_001" # Fine Dining 
restaurant_id = "restaurant_002" 

data = {
    "name": "Le Bernardin",
    "rating_1": 9,
    "rating_2": 8,
    "rating_3": 7,
    "notes": "Exceptional seafood tasting menu. Service was impeccable.",
    "visits": ["2023-08-12"],
    "location": "155 W 51st St, New York, NY 10019",
    "dishes": ["Tuna Tartare", "Lobster", "Chocolate-Olive Oil Dessert"],
    "photo": "https://www.le-bernardin.com/uploads/images/gallery/exterior.jpg"
}

new_restaurant = add_restaurant(category_id, restaurant_id, data)

